In [1]:
%pip install -r req.txt

Note: you may need to restart the kernel to use updated packages.


# Creating Neural Network
- Input Size: 1 x (2 x m), where m = number of regression models
- 4 models: Linear, Poly, XGBoost, Rand Forest, GPR

# Imports 

In [2]:
# core libraries
import pandas as pd 
import numpy as np

# deep learning
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

# preprocessing
from sklearn.model_selection import train_test_split

# Data Preprocessing

In [3]:
file = "sim_data_800.csv"
df = pd.read_csv(file)

# Check for missing values
print("Missing values:\n", df.isnull().sum())

# Check data types
print("Data types:\n", df.dtypes)

print(df.head())

Missing values:
 pred_1        0
conf_1        0
pred_2        0
conf_2        0
pred_3        0
conf_3        0
pred_4        0
conf_4        0
true_angle    0
dtype: int64
Data types:
 pred_1        float64
conf_1        float64
pred_2        float64
conf_2        float64
pred_3        float64
conf_3        float64
pred_4        float64
conf_4        float64
true_angle    float64
dtype: object
       pred_1    conf_1      pred_2    conf_2      pred_3    conf_3  \
0  164.444042  2.864122  130.483849  2.462217  113.406224  2.032592   
1   89.663341  3.107914   51.598905  2.151310   15.101633  0.735244   
2   18.679448  4.158178    6.997265  3.465243   49.060395  4.421956   
3   93.175008  2.218811  101.574192  1.015398  179.458291  2.415001   
4   65.501893  4.249734   81.768730  3.738013   19.341579  4.307536   

       pred_4    conf_4  true_angle  
0   54.208624  3.102034   48.684877  
1   45.622283  0.834111   34.586665  
2   87.298611  1.875094   55.343711  
3   62.681515  3.69594

# Data Prep

In [4]:
X = df.drop(columns = ['true_angle'])
Y = df['true_angle']

# 70/15/15 train val test split
X_train, X_temp, y_train, y_temp = train_test_split(X, Y, test_size=0.3, random_state=42) # 70% for training
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size = 0.5) # 15% each for val and test

# Model Architecture
- 2 hidden layer MLP

In [5]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size),
        )

    def forward(self, x):
        return self.net(x)
    

In [6]:
model = MLP(input_size=8, hidden_size=32, output_size=1)

# Training

In [7]:
# check for GPU availability
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print("Using device:", device)
model.to(device)

Using device: cuda


MLP(
  (net): Sequential(
    (0): Linear(in_features=8, out_features=32, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=32, out_features=32, bias=True)
    (4): ReLU()
    (5): Linear(in_features=32, out_features=1, bias=True)
  )
)

In [8]:
# convert training dataframes values into tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1) # returns y_tensor to size 1

# wrap into a dataset
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)

# create training batches
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [9]:
# convert validation dataframes values into tensors
X_val_tensor = torch.tensor(X_val.values, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32).unsqueeze(1) # returns y_tensor to size 1

# wrap into a dataset
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

# create val batches
val_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [10]:
# initalize optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001, weight_decay=1e-4)
loss_func = nn.MSELoss()

In [27]:
num_epochs = 30
best_val_loss = float('inf')
patience_limit = 20

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for X_batch, y_batch in train_loader:
        # move data to GPU or CPU
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        # forward step
        outputs = model(X_batch)
        loss = loss_func(outputs, y_batch)

        # backward step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # calculate running loss
        running_loss += loss.item() * X_batch.size(0)

    # total training loss
    train_loss = running_loss / len(train_loader.dataset)


    # validation phase
    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for X_val_batch, y_val_batch in val_loader:
            # move data to GPU or CPU
            X_val_batch = X_batch.to(device)
            y_val_batch = y_batch.to(device)

            # forward step
            val_outputs = model(X_val_batch)
            loss = loss_func(val_outputs, y_val_batch)

            # calculate running loss 
            val_loss += loss.item() * X_val_batch.size(0)

    val_loss /= len(val_loader.dataset)


    # adding early stopping to prevent overfitting            
    if best_val_loss > val_loss:
        best_val_loss = val_loss 
        best_model_state = model.state_dict()
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience_limit:
            print(f"Early stopping at epoch {epoch+1}")
            break

        
    print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Patience: {patience_counter}")

Epoch 1/30 | Train Loss: 79.7836 | Val Loss: 51.1696 | Patience: 0
Epoch 2/30 | Train Loss: 73.6470 | Val Loss: 47.1021 | Patience: 0
Epoch 3/30 | Train Loss: 77.4413 | Val Loss: 51.6684 | Patience: 1
Epoch 4/30 | Train Loss: 74.6365 | Val Loss: 67.0147 | Patience: 2
Epoch 5/30 | Train Loss: 78.4280 | Val Loss: 65.7503 | Patience: 3
Epoch 6/30 | Train Loss: 79.1564 | Val Loss: 56.1318 | Patience: 4
Epoch 7/30 | Train Loss: 77.9694 | Val Loss: 61.6236 | Patience: 5
Epoch 8/30 | Train Loss: 78.9431 | Val Loss: 39.5529 | Patience: 0
Epoch 9/30 | Train Loss: 78.2599 | Val Loss: 39.5244 | Patience: 0
Epoch 10/30 | Train Loss: 75.2618 | Val Loss: 77.7461 | Patience: 1
Epoch 11/30 | Train Loss: 76.6553 | Val Loss: 30.2365 | Patience: 0
Epoch 12/30 | Train Loss: 77.5132 | Val Loss: 54.9564 | Patience: 1
Epoch 13/30 | Train Loss: 75.2029 | Val Loss: 61.9546 | Patience: 2
Epoch 14/30 | Train Loss: 76.7574 | Val Loss: 67.0529 | Patience: 3
Epoch 15/30 | Train Loss: 77.4838 | Val Loss: 28.9979 | P

In [ ]:
# load best model after training
if best_model_state is not None:
    model.load_state_dict(best_model_state)